# Probabilistic tf-idf

Probabilistic tf-idf is a probabilistic model for matching large lists of sparse binary vectors - primarily, I had tokenized company names in mind when creating it. It extends the commonly used tf-idf method of matching tokenized names in that

 * In the absence of training data (i.e. no known matches), it produces scores similar to tf-idf.
 * Knowledge derived from known matches can be incorporated to learn both better entity descriptions as well as token-level relevance parameters.
 * Even without training data, it offers unsupervised training that allows to adapt to the matching task at hand to some extent.
 
The matching algorithm is computationally efficient by leveraging sparsity in the same way as tf-idf does.


# How it works


## Model

The model consists of _entities_ that are probability distributions over binary vectors of a fixed size - the _observations_. I call the elements of the vectors _tokens_ because the main application I have in mind is matching tokenized short strings, e.g. names. The distributions are independent Bernoulli distributions for the tokens. Each observation is assigned to an entity, but the number of entities is not necessarily fixed. In other words, the model is a non-parametric mixture with a product of independent Bernoullis as observation model.

With [notation as below](#Notation), the model is

$$
\begin{align}
\alpha_t &= \pi_t s_t \\
\beta_t &= (1 - \pi_t) s_t \\
r_{zt} &\sim \text{Beta}(\alpha_t, \beta_t) \\
p(\mathbf{x} \mid z) &= \prod_t r_{zt}^{x_t} \, (1 - r_{zt})^{1 - x_t}
\end{align}
$$

which can be complemented by an additional prior for $\alpha_t, \beta_t$, as discussed in the section on model training. In order to complete the mixture specification, one needs a partition prior as well. This is not in the scope of this document.


In the entity resolution settings I have in mind, entities usually have very few observations assigned to them (often just one), so that a lot of the information resides in the priors. Sharing parameters across entities allows to share information on token usage across entities.

## Intuition 

This section contains simplified expressions for matching scores in the entity resolution setting, where most entities have only one observation and token vectors are very sparse - e.g. containing a handful of tokens out of tens or hundreds of thousands per entity. This implies $\pi_t \ll 1$.

When matching entities, we consider the log-probability of observing a vector $\mathbf{x}$ for entity $z$ with count vector $\mathbf{k}_z$ and number of observations $n_z$. We set $n_z = 1$, so that $k_{zt} \in \{0, 1\}$. Since we are dealing with single entities and tokens only, we suppress the entity and token indices in the following and write $x, k, n$ etc.

### Interpretation of $s_t$

The probability to observe a token (i.e. $x = 1$) given that it has been observed for the entity (i.e. $k = 1$) is

$$
\begin{align}
p(x = 1 \mid k = 1, n = 1)
    &= \frac{s \pi + 1}{s + 1} \\
    &\approx \frac{1}{s + 1}
\end{align}
$$

where the second line holds if $\pi s \ll 1$, which is typically the case. Hence $\log s^{-1}$ are the logits for observing a token in a new observation given that it has been observed before, and is a measure of the reliability of a token. 

### Matching scores

Since both $k$ and $x$ can take on two values each, there are only four cases that can occur. We continue to assume $\pi s \ll 1$ and divide probabilities by prior probabilities to simplify expressions. Since the prior does not depend on the entity, this does not affect score differences.

For a token-entity match, $x = k = 1$, we obtain a score

$$
\begin{align}
\log\frac{p(x = 1 \mid k = 1, n=1)}{p_0(x = 1)} &= \log\pi^{-1} + \log\frac{\pi s + 1}{s + 1} \\
&\approx \log \pi^{-1} - \log\left(s + 1\right)
\end{align}
$$

whereas for non-matching we get

$$
\log \frac{p(x = 0 \mid k = 1, n=1)}{p_0(x=0)} = \log s - \log\left(s + 1\right)
$$

The same result holds for $x=0, k=1$. 

Finally matching absence of a token yields

$$
\begin{align}
\log\frac{p(x = 0 \mid k = 0, n=1)}{p_0(x = 0)} &= \log\left(1 - \pi\right)^{-1} + \log\frac{\left(1 - \pi\right) s + 1}{s + 1} \\
&\approx -\log \left(s + 1\right)
\end{align}
$$

Combining all parts leads to the score

$$
\log \frac{p(\mathbf{x} \mid z)}{p_0(\mathbf{x})} \approx
    \underbrace{\sum_{t \in \mathbf{x}\cap\mathbf{k}_z} \log\pi_t^{-1}}_{\text{matches}}
    - \underbrace{\sum_{t \in \mathbf{x}\cup\mathbf{k}_z \setminus \mathbf{x}\cap\mathbf{k}_z} \log s_t^{-1}}_{\text{mismatches}} - \underbrace{\sum_t\log\left(s_t + 1\right)}_{\text{const}}
$$


## Sparse Collapsed Formulation

An immediate problem with the formulation above is that the $r_{zt}$ are dense, so that even for tasks of moderate size memory issues arise. This can be fixed by marginalizing $r_{zt}$ using conjugacy. The resulting token distribution is still Bernoulli, but is expressed in terms of the sparse sufficient statistics

 * $n_z$ number of observations assigned to entity $z$ and
 * $\mathbf{k}_z$, the token count vector for all observations assigned to entity $z$

By standard calculations, the result is

$$
\begin{align}
p_{zt} &\equiv p(x_t = 1 \mid z) \\
    &= \frac{\alpha_t + k_{zt}}{\alpha_t + \beta_t + n_z}
\end{align}
$$

This sparse expression allows to sparsify the entire computation of the observation probabilies by shuffling terms. It is easier to do this for the log-probability

$$
\log p(\mathbf{x} \mid z) = \sum_t \left[x_t \log p_{zt} + (1 - x_t) \log(1 - p_{zt})\right]
$$

The basic idea is to shuffle terms such that sums containing specific (token and observation/entity dependent) data are restricted as much as possible, while unrestricted sums only contain less specific data. This allows to both reuse computations as well as save memory.

Let's denote the token probability with $k_{zt}$ set to zero (regardless of its actual value) by $p^0_{zt}$. Note that $p^0_{zt}$ depends on the entity through $n_z$ only. To simplify notation for sums, I use $t \in \mathbf{k}_z$ as shorthand for $k_{zt} > 0$ (and similar for other set operations).

Since $p_{zt} = p^0_{zt}$ if $t \notin \mathbf{k}_z$, we have

$$
\sum_{t} \log p_{zt} = \sum_{t\in \mathbf{k}_z} \log\frac{p_{zt}}{p^0_{zt}} + \sum_{t} \log p^0_{zt}
$$

Applying this to the observation probability, we get

$$
\begin{align}
\log p(\mathbf{x} \mid z) &= \sum_{t \in \mathbf{x}} \log p_{zt} +  \sum_{t \notin \mathbf{x}} \log (1 - p_{zt}) \\
&= \sum_{t \in \mathbf{x}\cap\mathbf{k}_z} \log \frac{p_{zt}}{p^0_{zt}} +
   \sum_{t\in \mathbf{x}} \log p^0_{zk} +
   \sum_{t \in \mathbf{k}_z \setminus \mathbf{x}} \log\frac{1 - p_{zt}}{1 - p^0_{zt}} +    
   \sum_{t\notin \mathbf{x}} \log \left(1 - p^0_{zt}\right) \\
&= \sum_{t \in \mathbf{x}\cap \mathbf{k}_z} \left[\log \frac{p_{zt}}{p^0_{zt}} - \log\frac{1 - p_{zt}}{1 - p^0_{zt}}\right] + \sum_{t \in \mathbf{k}_z} \log\frac{1 - p_{zt}}{1 - p^0_{zt}} + \sum_{t \in \mathbf{x}} \log\frac{p^0_{zt}}{1 - p^0_{zt}} + \sum_t \log \left(1 - p^0_{zt}\right)
\end{align}
$$


While this looks more complicated, it makes computation practical if we choose not to compute probabilities where the token overlap is zero.

1. The first sum over $\mathbf{x} \cap \mathbf{k}_z$ is a sparse matrix multiplication of two matrices with the same sparsity patterns as the document-term matrices for observations/entities have. Since we set probabilities for no token overlap to zero, the remaining terms have to be evaluated only if $\mathbf{x}\cap\mathbf{k}_z$ is not empty, i.e. for the non-zero entries of the result of the sparse matrix multiplication.
1. The second and last term can be pre-computed since they do not depend on the observation,
1. The last two terms depend on $z$ only through the number of known name variants $n_z$ because this is the only dependency that $p^0_{zt}$ has on $z$. Usually there will be only a handful of different values of $n_z$, way fewer than values of $z$. 
1. $p^0_{zt}$ can be looked up from a dense matrix of size (number of tokens, number of distinct $n_z$). If there are very many observations per entity (say thousands) and a large vocabulary (say hundreds of thousands) caching may no longer be feasible in memory, but this should rarely be a problem in practice.


## Training

__TODO__

# Appendix


## Notation

 * $t$ is used as index for tokens.
 * $z$ is used as index for entities.
 * $\mathbf{x}$ denotes an observation of tokens.
 * $n_z$ is the number of observations assigned to entity $z$.
 * $k_{zt}$ is the number of times token $t$ appears in all observations assigned to entity $z$. I also use $\mathbf{k}_z$ to denote the count vector for entity $z$.
 * $t \in \mathbf{x}$ means $x_t = 1$.
 * $r_{zt}$ is the Bernoulli rate for entity $z$ to emit token $t$
 * $\alpha_t, \beta_t$ are prior parameters for $r_{zt}$.
 * $\pi_t, s_t$ are an alternative mean/strength parametrization for $\alpha_t, \beta_t$
 * $p_{zt}$ is the probability for entity $z$ to emit token $t$ after marginalizing $r_{zt}$
 * $p^0_{zt}$ has the same meaning, but for the case of forcing $k_{zt} = 0$ regardless of its actual value


## Corrections for excluded observations (currently not implemented)

Setting probabilities for observations with no token overlap to zero leads to observation probabilities that do not sum to one. The probability for such observations is

$$
p(|\mathbf{x}\cap\mathbf{k}_z| = 0 \mid z) = \prod_{t \in \mathbf{k}_z} (1 - p_{zt})
$$

Since for the unrestricted case $\sum_{\mathbf{x}} p(\mathbf{x} \mid z) = 1$, we can compute normalized probabilities by setting

$$
\log p(\mathbf{x} \mid z) \rightarrow \log p(\mathbf{x} \mid z) - \log\left(1 - \prod_{t\in\mathbf{k}_z} (1 - p_{zt})\right)
$$

This correction is typically very small because the $t \in \mathbf{k}_z$ have high probability to occur, except when there is only one token, where a typical value is $-\log 0.8 \approx 0.2$

Similarly, we can correct the prior probabilities by disallowing observations with no tokens:

$$
\log p_0(\mathbf{x}) \rightarrow \log p_0(\mathbf{x}) - \log\left(1 - \prod_t (1 - \pi_t)\right)
$$

These corrections do not interfere with marginalizing $r_{zt}$ for predictions, but strictly speaking they do affect parameter training - even though the effect should be minor.